In [ ]:
def infix_to_postfix(expr): #수식 계산을 위한 후위 표시 전환 함수 
    stack = []
    priority = {'(': 0, '+': 1, '-': 1, '*': 2, '/': 2}
    postfix = []

    for token in expr:
        if token.isdigit() or token.isalpha(): # 문자나 연산자이면 바로 출력 
            postfix.append(token)
        elif token == '(': # ( 시작 시  우선 계산 위해 스택에 저장 
            stack.append(token)
        elif token == ')': # ) 끝날 시 우선 계산 할 것들 전부 POP해서 추가 
            while stack[-1] != '(':
                postfix.append(stack.pop())
            stack.pop()  # '(' 제거
        else: # 연산자인 경우
            while stack and priority.get(stack[-1], 0) >= priority.get(token, 0) and stack[-1] != '(': # 연산자 스택이 차 있으면서  
                postfix.append(stack.pop()) # 현재 연산자가 스택의 맨 위 보다 우선 순위가 낮은 경우 그리고 맨 위가 ( 가 아닌 경우 스택 연산자 먼저 출력 
            stack.append(token) # 모든 연산자는 일단 스택에 저장 

    while stack: # 스택 털기 
        postfix.append(stack.pop())

    return postfix # 연산 순서 반환 

def is_binary_string(s): # 바이너리 검사 함수 
    return all(map(lambda c: c in ['0', '1'], s))

def returnadd(var): # 올바른 주소 지정위한 함수
    while True:
      address = input(f'{var}의 메모리 위치를 지정해주세요 : ')
      if is_binary_string(address) and len(address) == 4: # 현재 메모리는 16비트 이외 전부 쳐냄 
        return address
      else:
        print('잘못된 주소입니다.')

def returnbinval(var): # 올바른 바이너리 값 지정위한 함수
    bin_var = input(f'{var}의 할당할 값을 지정해주세요 : ')
    if is_binary_string(bin_var):
      return bin_var
    else: # 혹시 숫자 적었으면 이진 코드로 바꿔주기 
      print("숫자가 이진변환 되어 삽입됩니다. {} -> {}".format(var,var,bin(bin_var)[2:]))
      return bin(bin_var)[2:]

def get_type(element): # 요소 타입 알아내는 함수 이 함수는 숫자, 변수, 연산자, 이외의 값으로 구분 
    if element.isdigit():
        return 'number'
    elif element.isalpha():
        return 'alphabet'
    elif element in ['+', '-', '*', '/']:
        return 'operator'
    else:
        return 'unknown'

def exp_2_code(): # 수식을 pl 코드로 변환 하기 

  memory = {} # 변수의 주소 : 변수가 가질 값 
  mapping_add = {} # 변수의 이름 : 변수의 주소 
  plcodes = [] # 실제 코드가 될 리스트 

  while True:
    expr = input("수식을 입력해 주세요 :") # 실 사용 코드 
    #expr = 'Z = A + 3 * B - C / 8' # 예시문 
    if '=' not in expr:
      print('등호(대입연산자)가 없습니다.')
      if input('뒤로 가시겠습니까?(y/n) : ').strip().lower() == 'y':
        return
    else:
      expr = expr.split('=') # 대입 연산자 기준 좌우변 분리 
      equal_left = expr[0].split()[0] # 좌변 
      if len(equal_left) == 1 and get_type(equal_left) == 'alphabet': # 대입 연산자의 좌변 변수,1개인 것 확인  
        equal_right = infix_to_postfix(list(expr[1].replace(' ',''))) # 우변 분리 후 후위 표기 방식으로 정렬
        #print(equal_left) # 좌변 확인용 
        #print(equal_right) # 우변 확인용 
        break # while 꺠기 
      else: # 대입 연산자의 좌변은 변수 1개
        print('대입연산자 죄측의 복수의 문자가 있습니다.') 

  vars = [char for char in equal_right if char.isalpha()] # 변수들만 뽑아내기 
  #print(vars) # 변수들 
  #operators = [token for token in equal_right if token in ['+', '-', '*', '/']] # 연산자들만 뽑아내기 (계산 순서 맞는지 확인용)
  #print(operators) # 연산자들 

  address = returnadd(equal_left) # 좌변 주소 값 할당 
  memory[address] = returnbinval(equal_left)
  mapping_add[equal_left] = address

  for var in vars: # 우변 주소 값 할당 
    while True:
      address = returnadd(var)
      if address in memory.keys(): # 주소가 이미 있는 경우 
        print("이미 주소가 할당 되었습니다.")
      else: # 새 주소라면 
        memory[address] = returnbinval(var)
        mapping_add[var] = address
        break # while 깨기 

  print(" ")
  print("파일 내용이 다음과 같이 작성됩니다.")
  print("="*40)
  #print(memory) # 현재 메모리 
  #print(mapping_add) # 주소 사상 값
  for add,val in memory.items(): # 전체 변수 값 할당 코드
    plcodes += [f"LOAD {val}", f"STOR {add}", " "]

  for i, element in enumerate(equal_right):
    now_type = get_type(element)

    if now_type == 'alphabet': # 변수가 들어오면
      plcodes.append(f"READ {mapping_add[element]}") # 변수의 주소를 DR에 읽어 온다 
      if i+1 < len(equal_right) and get_type(equal_right[i+1]) in ['alphabet', 'number']: # 뒤에 오는 요소의 타입이 처리 변수/숫자라면 스택에 PUSH
          plcodes += ["PUSH"," "]
      elif i+1 < len(equal_right) and get_type(equal_right[i+1]) == 'operator': # 연산자라면 계산 준비 
          plcodes.append(" ")

    elif now_type == 'number': # 숫자가 들어오면
      plcodes.append(f"LOAD {bin(int(element))[2:].zfill(4)}") # 숫자의 값을 DR에 읽어 오는 명령 추가 
      if i+1 < len(equal_right) and get_type(equal_right[i+1]) in ['alphabet', 'number']: # 뒤에 오는 요소의 타입이 처리 변수/숫자라면 스택에 PUSH
          plcodes += ["PUSH"," "]
      elif i+1 < len(equal_right) and get_type(equal_right[i+1]) == 'operator': # 연산자라면 계산 준비 
          plcodes.append(" ")

    elif now_type == 'operator': # 연산자가 들어오면
      if get_type(equal_right[i-1]) == 'operator': #전에도 연산자 였다면 
        plcodes.append("POP") # 스택에 있는 값을 DR로 불러오는 명령문 추가 

      if element == '+': # 각 연산자에 맞는 계산 명령문 추가 
        plcodes += ["ADD"," "]
      elif element == '-':
        plcodes += ["SUB"," "]
      elif element == '*':
        plcodes += ["MUL"," "]
      elif element == '/':
        plcodes += ["DIV"," "]

    if i+1 == len(equal_right): # 우변의 끝인 경우 좌변에 대입해 출력 
      plcodes.append("POP") # DR로 올리고 좌변 주소에 저장 후 출력 명령문 추가 
      plcodes.append(f"STOR {mapping_add[equal_left]}")
      plcodes.append(f"PRINT {mapping_add[equal_left]}")

  for line in plcodes: # 파일에 추가될 명령문들 확인용 출력 
    print(line)

  print("="*40)
  file_name = input("파일 이름만 적어주세요 : ") # 실제 파일 생성 
  with open(file_name + ".pl","w") as f:
    for line in plcodes: # plcodes에 있는 명령문들 파일에 쓰기 
      f.write(line+"\n")
  f.close()
  print(f"{file_name + '.pl'} 파일이 생성되었습니다.\n")

def pl2bi(file_name): # pl을 bi 파일로 변경하기 

  opcodes = {
        'LOAD': ['0000'],
        'READ': ['0001'],
        'STOR': ['0010'],
        'PUSH': ['0011','0100'],
        'POP': ['0101'],
        'ADD': ['0110'],
        'SUB': ['0111'],
        'MUL': ['1000'],
        'DIV': ['1001'],
        'PRINT': ['1010','1011'],
              } # 딕셔너리로 파싱

  write_code = [] # 쓸 바이너리 문장 리스트

  with open(file_name + ".pl","r") as f: # code1.pl 파일을 읽어 분석
      for line in f:
          bin_code = line.split() # opcode와 operand끼리 분해
        
          if len(bin_code) == 0: # 빈 줄 처리 
              write_code.append("")
        
          elif len(bin_code) == 1: # operand가 없는 경우 
              opcode = bin_code[0]

              if opcode in opcodes:
                write_code.append(opcodes[opcode][0])
              else:
                  write_code.append("1111 1111 1111") # 에러문 삽입 

          elif len(bin_code) == 2: # operand가 있는 경우 
              opcode = bin_code[0]
              operand = bin_code[1]

              if opcode == "PRINT" or opcode == "PUSH":
                  write_code.append(opcodes[opcode][1] + " " + operand)
              elif opcode in opcodes:
                  write_code.append(opcodes[opcode][0] + " " + operand)
              else:
                  write_code.append("1111 1111 1111") # 에러문 삽입 
        
          else:
              write_code.append("1111 1111 1111") # 에러문 삽입 
            

  f.close()

  # 확인용
  '''
  for line in parsed_code:
    print(line)
  '''

  # code.bi 생성
  with open(file_name + ".bi",'w') as f:
      for line in write_code:
          f.write(line+"\n")
  f.close()

  print("{} -> {} 변경 완료 되었습니다.\n".format(file_name + ".pl",file_name + ".bi"))

def bi2exe(file_name): # bi를 실제 실행하기 

  DR = 9999 # 데이터 레지스터
  MR = {} # 메모리
  ST = [] # 스택
  error = 0 # 에러 포함 유무

  def print_error(string,line_num):
    print(string)
    print("오류 발생 줄 : {}".format(line_num))


  with open(file_name + ".bi", "r") as f: # code2.bl 파일을 읽어 분석
      lines = f.readlines()

  machine_code = [] # 기계어 코드를 받음
  line_num = 0 # 현재 실행하는 줄

  print("========== 실행 내용 ==========")
  for line in lines:
      line = line.strip()
      if line == "": # 빈 행 스킵
        line_num += 1
        continue

      elif len(line.split()) > 2: # 현재 삼항 binary code는 오류문 외에 없음
        print_error("에러코드가 들어있습니다.", line_num)
        return
          
      opcode = line[:4] # opcode 슬라이싱
      operand = line[5:] # operand 슬라이싱

      if opcode == "0000": # LOAD NUM
          DR = int(operand,2) # DR에 2진값 10진화 해서 저장
      
      elif opcode == "0001": # READ [A]
          if operand in MR: # 주소(operand)에 있는지 검사 
            DR = MR[operand] # 주소(operand)에 있는 값을 DR에 저장

          else: # 주소(operand)로 할당한 값이 없을 때 오류 
            print_error("주소 : {}에 값이 없습니다.".format(operand),line_num)
            return
      
      elif opcode == "0010": # STOR [A]
            MR[operand] = DR # DR값을 주소(operand) : 값(DR)으로 MR에 딕셔너리 저장
      
      elif opcode == "0011": # PUSH
          ST.append(DR) # DR값을 ST 스택에 PUSH
      
      elif opcode == "0100": # PUSH NUM
          ST.append(int(operand,2)) # NUM(operand)값을 ST 스택에 PUSH
    
      elif opcode == "0101": # POP
          if len(ST) != 0: # 빈 스택 인지 확인
            DR = ST.pop() # 스택 맨 위 값 제거 및 DR에 저장
          
          else: # 빈 스택일 시 오류
            print_error("스택이 비었습니다.",line_num)
            return
      
      elif opcode == "0110": # ADD
          if len(ST) != 0: # 빈 스택 인지 확인
            tmp = ST.pop() # 스택 맨 위 값 + DR을 스택 맨 위로로
            tmp += DR
            ST.append(tmp)
          
          else: # 빈 스택일 시 오류
            print_error("스택이 비었습니다.",line_num)
            return
      
      elif opcode == "0111": # SUB
          if len(ST) != 0: # 빈 스택 인지 확인
            tmp = ST.pop() # 스택 맨 위 값 + DR을 스택 맨 위로로
            tmp -= DR
            ST.append(tmp)
          
          else: # 빈 스택일 시 오류
            print_error("스택이 비었습니다.",line_num)
            return
      
      elif opcode == "1000": # MUL
          if len(ST) != 0: # 빈 스택 인지 확인
            tmp = ST.pop() # 스택 맨 위 값 + DR을 스택 맨 위로로
            tmp *= DR
            ST.append(tmp)
          
          else: # 빈 스택일 시 오류
            print_error("스택이 비었습니다.",line_num)
            return
      
      elif opcode == "1001": # DIV
          if len(ST) != 0: # 빈 스택 인지 확인
            if DR != 0: # 나누는 값이 0이 아닐 때만
              tmp = ST.pop() # 스택 맨 위 값 + DR을 스택 맨 위로로
              tmp /= DR
              ST.append(int(tmp))
            else: #ZeroDivided 오류
              print_error("0으로 나누었습니다.",line_num)
              return
          
          else: # 빈 스택일 시 오류
            print_error("스택이 비었습니다.",line_num)
            return
      
      elif opcode == "1010": # PRINT
          print("{0:b}".zfill(4).format(DR,'b')) # 현재 DR 값 출력
          with open(file_name + ".txt",'w') as f: # txt 파일 생성
            f.write(bin(DR)[2:].zfill(4)+"\n")
          f.close()
    
      elif opcode == "1011": # PRINT [A]
          if operand in MR: # 주소(operand)에 있는지 검사 
            print("{0:b}".zfill(4).format(MR[operand])) # 주소에 있는 값 출력
            with open(file_name + ".txt",'w') as f: # txt 파일 생성
              f.write(bin(DR)[2:].zfill(4)+"\n")
            f.close()

          else: # 주소(operand)로 할당한 값이 없을 때 오류 
            print_error("주소 : {}에 값이 없습니다.".format(operand),line_num)
            return

      line_num += 1

  f.close()
  print("===============================")
  print("정상 실행 되었습니다.")
  print("총 실행한 코드줄 : {}\n".format(line_num+1))

def file_create(file_name,file_extension): # 확장자 명에 따른 파일 생성 함수 
  try:
    if file_extension =='pl': # pl이면 bi로 변경 
      pl2bi(file_name)
    elif file_extension =='bi': # bi이면 파일 내용 실행 
      bi2exe(file_name) 
    else: # 잘못된 확장자명 할당 
      print("확장자 명이 잘못 되었습니다.")
      return 1
  except FileNotFoundError: # 모든 오류는 파일을 찾을 수 없을 때 발생
    print("파일이 경로 상에 존재하지 않거나 파일 명을 다시 확인해 주십시오.")
  except Exception:
    print("미확인 오류")
  else:
    return 0
  end = input("뒤로 가시겠습니까?(y/n) : ").strip().lower()
  if end == 'y':
    return 0
  return 1


def exit_program(): # 종료문 함수 정의 
  while True:
      out = input("종료하시겠습니까? y/n : ").strip().lower()
      if out == 'y':
        return 1
      elif out == 'n':
        print(" ")
        return 0
      else:
        print("다시 입력해주세요.\n")
        continue

def print_file(file_full_name):
  print("{} 파일의 내용을 출력합니다.".format(file_full_name)) # 파일 내용 함수 
  print("="*40)
  with open(file_full_name,"r") as f:
    lines = f.readlines()
    for line in lines:
      print(line,end="")
  print("="*40+"\n")

#실행기 UI
print("="*40)
print("exp -> .pl / .pl -> .bi / .bi -> exe Program")
file_name = ""

exit = 0

while exit == 0:
  print("실행할 내용을 보기에서 선택해주세요.")
  print("1. exp -> .pl")
  print("2. .pl -> .bi")
  print("3. .bi -> exe")
  print("4. print file context")
  print("5. exit")
  select = input(">>> ").strip()
  repeater = 1

  if select == '1' : # 수식 -> .pl 파일 변환 실행 
    exp_2_code()

  elif select == '2': # .pl -> .bi 파일 변환 실행 
    while repeater:
      file_name = input("피일 명(ex:code1.pl)을 입력해주세요 : ")
      repeater = file_create(file_name[:-3],'pl')

  elif select == '3': # .bi -> 실행 파일 변환 실행 
    while repeater:
      file_name = input("피일 명(ex:code1.bi)을 입력해주세요 : ")
      repeater = file_create(file_name[:-3],'bi')
  
  elif select == '4': # 파일 내용 출력
    file_name = input("피일 명(ex:파일이름.확장자명)을 입력해주세요 : ") 
    print_file(file_name)

  elif select == '5': # 나가기 
    if exit_program() == 1:
      break
    else:
      continue
  else:
    print("보기 이외의 입력입니다.")

  exit = exit_program()

exp -> .pl / .pl -> .bi / .bi -> exe Program
실행할 내용을 보기에서 선택해주세요.
1. exp -> .pl
2. .pl -> .bi
3. .bi -> exe
4. print file context
5. exit
>>> 1
수식을 입력해 주세요 :Z = A + 3 * B - C / 8
Z의 메모리 위치를 지정해주세요 : 1101
Z의 할당할 값을 지정해주세요 : 0000
A의 메모리 위치를 지정해주세요 : 0100
A의 할당할 값을 지정해주세요 : 0010
B의 메모리 위치를 지정해주세요 : 1011
B의 할당할 값을 지정해주세요 : 0101
C의 메모리 위치를 지정해주세요 : 1100
C의 할당할 값을 지정해주세요 : 1000
 
파일 내용이 다음과 같이 작성됩니다.
LOAD 0000
STOR 1101
 
LOAD 0010
STOR 0100
 
LOAD 0101
STOR 1011
 
LOAD 1000
STOR 1100
 
READ 0100
PUSH
 
LOAD 0011
PUSH
 
READ 1011
 
MUL
 
POP
ADD
 
READ 1100
PUSH
 
LOAD 1000
 
DIV
 
POP
SUB
 
POP
STOR 1101
PRINT 1101
파일 이름만 적어주세요 : code
code.pl 파일이 생성되었습니다.

종료하시겠습니까? y/n : n
 
실행할 내용을 보기에서 선택해주세요.
1. exp -> .pl
2. .pl -> .bi
3. .bi -> exe
4. print file context
5. exit
>>> 2
피일 명(ex:code1.pl)을 입력해주세요 : code.pl
code.pl -> code.bi 변경 완료 되었습니다.

종료하시겠습니까? y/n : n
 
실행할 내용을 보기에서 선택해주세요.
1. exp -> .pl
2. .pl -> .bi
3. .bi -> exe
4. print file context
5. exit
>>> 3
피일 명(ex:code1.bi)을 입력해주세요 : code.